In [28]:
# import library
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb
import time

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

In [2]:
#load  data
df = pd.read_csv('../data/HCD35_neg_norm.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [3]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# add result to list 
o = []
o.append(round((clf.score(X_test, y_test))*100, 2))

In [4]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)


# result append to list
o.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.30137
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.22335
[3]	valid_0's multi_logloss: 1.15893
[4]	valid_0's multi_logloss: 1.09947
[5]	valid_0's multi_logloss: 1.04987
[6]	valid_0's multi_logloss: 1.01132
[7]	valid_0's multi_logloss: 0.977166
[8]	valid_0's multi_logloss: 0.949006
[9]	valid_0's multi_logloss: 0.923959
[10]	valid_0's multi_logloss: 0.902713
[11]	valid_0's multi_logloss: 0.880888
[12]	valid_0's multi_logloss: 0.861254
[13]	valid_0's multi_logloss: 0.840925
[14]	valid_0's multi_logloss: 0.82072
[15]	valid_0's multi_logloss: 0.805376
[16]	valid_0's multi_logloss: 0.788799
[17]	valid_0's multi_logloss: 0.776449
[18]	valid_0's multi_logloss: 0.765656
[19]	valid_0's multi_logloss: 0.754441
[20]	valid_0's multi_logloss: 0.740786
[21]	valid_0's multi_logloss: 0.73216
[22]	valid_0's multi_logloss: 0.725209
[23]	valid_0's multi_logloss: 0.713766
[24]	valid_0's multi_logloss: 0.706488
[25]	valid_0's multi

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [5]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [6]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
o.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)
o.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 2s 5ms/step - loss: 2.1360 - acc: 0.2413 - val_loss: 1.4381 - val_acc: 0.3511
Epoch 2/100
373/373 [==============================] - 0s 221us/step - loss: 1.9896 - acc: 0.2708 - val_loss: 1.3772 - val_acc: 0.3617
Epoch 3/100
373/373 [==============================] - 0s 222us/step - loss: 1.9666 - acc: 0.2842 - val_loss: 1.3276 - val_acc: 0.3723
Epoch 4/100
373/373 [==============================] - 0s 213us/step - loss: 1.7529 - acc: 0.3485 - val_loss: 1.2927 - val_acc: 0.4255
Epoch 5/100
373/373 [==============================] - 0s 206us/step - loss: 1.6890 - acc: 0.3137 - val_loss: 1.2828 - val_acc: 0.3617
Epoch 6/100
373/373 [==============================] - 0s 220us/step - loss: 1.6320 - acc: 0.3271 - val_loss: 1.2864 - val_acc: 0.3830
Epoch 7/100
373/373 [==============================] - 0s 204us/step - loss: 1.5200 - acc: 0.3861 - val_loss: 1.2804 - val_acc: 0.4043
Epoch 8/100


In [10]:
g = []
g.append(o)

In [11]:
# o = {}

In [12]:
# o['xgb+rf'] = [round((sum(np.argmax((xgb_result+rf_result) , axis=1) == y_test)/len(y_test))*100, 2)]
# o['xgb+lgb'] = [round((sum(np.argmax((xgb_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
# o['rf+lgb'] = [round((sum(np.argmax((rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
# o['xgb+rf+lgb'] = [round((sum(np.argmax((xgb_result+rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

In [13]:
#load  data
df = pd.read_csv('../data/HCD45_neg_norm.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [14]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# add result to list 
o = []
o.append(round((clf.score(X_test, y_test))*100, 2))

In [15]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)


# result append to list
o.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.30728
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.24428
[3]	valid_0's multi_logloss: 1.19091
[4]	valid_0's multi_logloss: 1.14783
[5]	valid_0's multi_logloss: 1.1074
[6]	valid_0's multi_logloss: 1.07527
[7]	valid_0's multi_logloss: 1.04467
[8]	valid_0's multi_logloss: 1.01698
[9]	valid_0's multi_logloss: 0.996891
[10]	valid_0's multi_logloss: 0.975222
[11]	valid_0's multi_logloss: 0.948461
[12]	valid_0's multi_logloss: 0.927557
[13]	valid_0's multi_logloss: 0.908382
[14]	valid_0's multi_logloss: 0.890571
[15]	valid_0's multi_logloss: 0.874068
[16]	valid_0's multi_logloss: 0.858673
[17]	valid_0's multi_logloss: 0.842187
[18]	valid_0's multi_logloss: 0.833464
[19]	valid_0's multi_logloss: 0.822464
[20]	valid_0's multi_logloss: 0.814508
[21]	valid_0's multi_logloss: 0.804284
[22]	valid_0's multi_logloss: 0.796206
[23]	valid_0's multi_logloss: 0.787263
[24]	valid_0's multi_logloss: 0.779908
[25]	valid_0's multi_

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [17]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
o.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [18]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)
o.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 2s 5ms/step - loss: 2.3167 - acc: 0.2359 - val_loss: 1.6018 - val_acc: 0.3298
Epoch 2/100
373/373 [==============================] - 0s 397us/step - loss: 1.8618 - acc: 0.3190 - val_loss: 1.5464 - val_acc: 0.3298
Epoch 3/100
373/373 [==============================] - 0s 360us/step - loss: 1.8906 - acc: 0.3083 - val_loss: 1.5075 - val_acc: 0.3298
Epoch 4/100
373/373 [==============================] - 0s 403us/step - loss: 1.8461 - acc: 0.3164 - val_loss: 1.4650 - val_acc: 0.3511
Epoch 5/100
373/373 [==============================] - 0s 363us/step - loss: 1.8787 - acc: 0.3351 - val_loss: 1.4279 - val_acc: 0.3298
Epoch 6/100
373/373 [==============================] - 0s 388us/step - loss: 1.7753 - acc: 0.3512 - val_loss: 1.3920 - val_acc: 0.3404
Epoch 7/100
373/373 [==============================] - 0s 368us/step - loss: 1.5961 - acc: 0.3539 - val_loss: 1.3544 - val_acc: 0.3404
Epoch 8/100


In [19]:
g.append(o)

In [20]:
#load  data
df = pd.read_csv('../data/HCD65_neg_norm.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [21]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# add result to list 
o = []
o.append(round((clf.score(X_test, y_test))*100, 2))

In [22]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)


# result append to list
o.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.31383
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.25405
[3]	valid_0's multi_logloss: 1.20084
[4]	valid_0's multi_logloss: 1.15406
[5]	valid_0's multi_logloss: 1.1178
[6]	valid_0's multi_logloss: 1.0844
[7]	valid_0's multi_logloss: 1.05432
[8]	valid_0's multi_logloss: 1.02851
[9]	valid_0's multi_logloss: 1.00402
[10]	valid_0's multi_logloss: 0.980917
[11]	valid_0's multi_logloss: 0.959213
[12]	valid_0's multi_logloss: 0.947037
[13]	valid_0's multi_logloss: 0.93271
[14]	valid_0's multi_logloss: 0.92061
[15]	valid_0's multi_logloss: 0.909448
[16]	valid_0's multi_logloss: 0.895203
[17]	valid_0's multi_logloss: 0.885169
[18]	valid_0's multi_logloss: 0.87417
[19]	valid_0's multi_logloss: 0.86671
[20]	valid_0's multi_logloss: 0.86169
[21]	valid_0's multi_logloss: 0.856482
[22]	valid_0's multi_logloss: 0.849996
[23]	valid_0's multi_logloss: 0.844023
[24]	valid_0's multi_logloss: 0.83711
[25]	valid_0's multi_logloss:

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [24]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
o.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [25]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)
o.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 2s 5ms/step - loss: 2.3458 - acc: 0.2359 - val_loss: 1.6102 - val_acc: 0.3511
Epoch 2/100
373/373 [==============================] - 0s 505us/step - loss: 1.8565 - acc: 0.3512 - val_loss: 1.6563 - val_acc: 0.3085
Epoch 3/100
373/373 [==============================] - 0s 481us/step - loss: 1.9349 - acc: 0.3405 - val_loss: 1.7194 - val_acc: 0.2872
Epoch 4/100
373/373 [==============================] - 0s 493us/step - loss: 1.6329 - acc: 0.3834 - val_loss: 1.7205 - val_acc: 0.2979
Epoch 5/100
373/373 [==============================] - 0s 534us/step - loss: 1.9863 - acc: 0.3566 - val_loss: 1.7068 - val_acc: 0.2979
Epoch 6/100
373/373 [==============================] - 0s 544us/step - loss: 1.7797 - acc: 0.3592 - val_loss: 1.6782 - val_acc: 0.2979
Epoch 7/100
373/373 [==============================] - 0s 502us/step - loss: 1.6674 - acc: 0.3968 - val_loss: 1.6260 - val_acc: 0.2872
Epoch 8/100


In [26]:
g.append(o)

In [29]:
pd.DataFrame(
    g, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'], 
    index=['HCD35', 'HCD45', 'HCD65']
).to_csv('../result/norm_neg.csv')

In [30]:
#load  data
df = pd.read_csv('../data/HCD35_pos_norm.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [31]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# add result to list 
o = []
o.append(round((clf.score(X_test, y_test))*100, 2))

In [32]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)


# result append to list
o.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.29145
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.21452
[3]	valid_0's multi_logloss: 1.15349
[4]	valid_0's multi_logloss: 1.09671
[5]	valid_0's multi_logloss: 1.04309
[6]	valid_0's multi_logloss: 0.998792
[7]	valid_0's multi_logloss: 0.958398
[8]	valid_0's multi_logloss: 0.920585
[9]	valid_0's multi_logloss: 0.884814
[10]	valid_0's multi_logloss: 0.853752
[11]	valid_0's multi_logloss: 0.826709
[12]	valid_0's multi_logloss: 0.801109
[13]	valid_0's multi_logloss: 0.776881
[14]	valid_0's multi_logloss: 0.754607
[15]	valid_0's multi_logloss: 0.732409
[16]	valid_0's multi_logloss: 0.713332
[17]	valid_0's multi_logloss: 0.695356
[18]	valid_0's multi_logloss: 0.677332
[19]	valid_0's multi_logloss: 0.662276
[20]	valid_0's multi_logloss: 0.64722
[21]	valid_0's multi_logloss: 0.635049
[22]	valid_0's multi_logloss: 0.621298
[23]	valid_0's multi_logloss: 0.610937
[24]	valid_0's multi_logloss: 0.600643
[25]	valid_0's mul

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [33]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [34]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
o.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [35]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)


Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 3s 2ms/step - loss: 2.2794 - acc: 0.2500 - val_loss: 1.5373 - val_acc: 0.2619
Epoch 2/100
1340/1340 [==============================] - 1s 824us/step - loss: 1.8903 - acc: 0.2769 - val_loss: 1.4514 - val_acc: 0.2857
Epoch 3/100
1340/1340 [==============================] - 1s 815us/step - loss: 1.7645 - acc: 0.3060 - val_loss: 1.3951 - val_acc: 0.3065
Epoch 4/100
1340/1340 [==============================] - 1s 799us/step - loss: 1.6725 - acc: 0.3373 - val_loss: 1.3386 - val_acc: 0.3512
Epoch 5/100
1340/1340 [==============================] - 1s 812us/step - loss: 1.6116 - acc: 0.3537 - val_loss: 1.2636 - val_acc: 0.4167
Epoch 6/100
1340/1340 [==============================] - 1s 829us/step - loss: 1.5758 - acc: 0.3776 - val_loss: 1.2247 - val_acc: 0.4494
Epoch 7/100
1340/1340 [==============================] - 1s 817us/step - loss: 1.4550 - acc: 0.4358 - val_loss: 1.2062 - val_acc: 0.4

NameError: name 't' is not defined

In [36]:
o.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

In [37]:
g = []
g.append(o)
g

[[75.0, 83.93, 78.57, 0.6845238095238095]]

In [38]:
#load  data
df = pd.read_csv('../data/HCD45_pos_norm.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [39]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# add result to list 
o = []
o.append(round((clf.score(X_test, y_test))*100, 2))

In [40]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)


# result append to list
o.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.29231
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.22044
[3]	valid_0's multi_logloss: 1.15759
[4]	valid_0's multi_logloss: 1.10527
[5]	valid_0's multi_logloss: 1.05431
[6]	valid_0's multi_logloss: 1.01236
[7]	valid_0's multi_logloss: 0.971375
[8]	valid_0's multi_logloss: 0.935873
[9]	valid_0's multi_logloss: 0.904783
[10]	valid_0's multi_logloss: 0.874381
[11]	valid_0's multi_logloss: 0.846309
[12]	valid_0's multi_logloss: 0.822394
[13]	valid_0's multi_logloss: 0.798726
[14]	valid_0's multi_logloss: 0.777977
[15]	valid_0's multi_logloss: 0.755864
[16]	valid_0's multi_logloss: 0.737617
[17]	valid_0's multi_logloss: 0.721305
[18]	valid_0's multi_logloss: 0.706719
[19]	valid_0's multi_logloss: 0.690823
[20]	valid_0's multi_logloss: 0.678504
[21]	valid_0's multi_logloss: 0.663004
[22]	valid_0's multi_logloss: 0.650599
[23]	valid_0's multi_logloss: 0.639505
[24]	valid_0's multi_logloss: 0.630802
[25]	valid_0's mul

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [41]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [42]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
o.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [43]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)
o.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 3s 2ms/step - loss: 2.2043 - acc: 0.2507 - val_loss: 1.5073 - val_acc: 0.2768
Epoch 2/100
1340/1340 [==============================] - 1s 854us/step - loss: 2.0194 - acc: 0.2881 - val_loss: 1.5037 - val_acc: 0.2887
Epoch 3/100
1340/1340 [==============================] - 1s 854us/step - loss: 1.8062 - acc: 0.3149 - val_loss: 1.5428 - val_acc: 0.3095
Epoch 4/100
1340/1340 [==============================] - 1s 877us/step - loss: 1.8060 - acc: 0.3433 - val_loss: 1.5150 - val_acc: 0.3155
Epoch 5/100
1340/1340 [==============================] - 1s 953us/step - loss: 1.7186 - acc: 0.3657 - val_loss: 1.4256 - val_acc: 0.3274
Epoch 6/100
1340/1340 [==============================] - 1s 898us/step - loss: 1.5927 - acc: 0.3761 - val_loss: 1.3668 - val_acc: 0.3363
Epoch 7/100
1340/1340 [==============================] - 1s 860us/step - loss: 1.5062 - acc: 0.4179 - val_loss: 1.3188 - val_acc: 0.3

In [44]:
g.append(o)

In [45]:
#load  data
df = pd.read_csv('../data/HCD65_pos_norm.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [46]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# add result to list 
o = []
o.append(round((clf.score(X_test, y_test))*100, 2))

In [47]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)


# result append to list
o.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.2955
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.22148
[3]	valid_0's multi_logloss: 1.16028
[4]	valid_0's multi_logloss: 1.10455
[5]	valid_0's multi_logloss: 1.05653
[6]	valid_0's multi_logloss: 1.01408
[7]	valid_0's multi_logloss: 0.970765
[8]	valid_0's multi_logloss: 0.93398
[9]	valid_0's multi_logloss: 0.903181
[10]	valid_0's multi_logloss: 0.872622
[11]	valid_0's multi_logloss: 0.845132
[12]	valid_0's multi_logloss: 0.823578
[13]	valid_0's multi_logloss: 0.802204
[14]	valid_0's multi_logloss: 0.780579
[15]	valid_0's multi_logloss: 0.760263
[16]	valid_0's multi_logloss: 0.745311
[17]	valid_0's multi_logloss: 0.730848
[18]	valid_0's multi_logloss: 0.715726
[19]	valid_0's multi_logloss: 0.702982
[20]	valid_0's multi_logloss: 0.691537
[21]	valid_0's multi_logloss: 0.677025
[22]	valid_0's multi_logloss: 0.666607
[23]	valid_0's multi_logloss: 0.654698
[24]	valid_0's multi_logloss: 0.647321
[25]	valid_0's multi

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [48]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [49]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
o.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [50]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)
o.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 3s 3ms/step - loss: 2.2147 - acc: 0.2507 - val_loss: 1.6337 - val_acc: 0.2619
Epoch 2/100
1340/1340 [==============================] - 1s 869us/step - loss: 1.8679 - acc: 0.3254 - val_loss: 1.7067 - val_acc: 0.2708
Epoch 3/100
1340/1340 [==============================] - 1s 898us/step - loss: 1.8764 - acc: 0.3276 - val_loss: 1.6233 - val_acc: 0.2827
Epoch 4/100
1340/1340 [==============================] - 1s 917us/step - loss: 1.7233 - acc: 0.3575 - val_loss: 1.5004 - val_acc: 0.3065
Epoch 5/100
1340/1340 [==============================] - 1s 918us/step - loss: 1.6823 - acc: 0.3657 - val_loss: 1.4325 - val_acc: 0.2887
Epoch 6/100
1340/1340 [==============================] - 1s 865us/step - loss: 1.6049 - acc: 0.3866 - val_loss: 1.4134 - val_acc: 0.3333
Epoch 7/100
1340/1340 [==============================] - 1s 906us/step - loss: 1.5064 - acc: 0.3888 - val_loss: 1.3960 - val_acc: 0.3

In [51]:
g.append(o)

In [52]:
pd.DataFrame(
    g, 
    columns=['Random Forest', 'LightGBM', 'XGBoost','Keras'], 
    index=['HCD35', 'HCD45', 'HCD65']
).to_csv('../result/norm_pos.csv')